In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url_items = ["womens-belts","womens-hats-caps","hair-accessories-8941",\
             "womens-sunglasses","womens-scarves-wraps",\
             "womens-wallets-card-cases-money-organizers"]

In [3]:
for  url_item in url_items:

   belt_accessories_url = f'https://www.jumia.com.ng/{url_item}/?page=2#catalog-listing'
   belt_list_url = f'https://www.jumia.com.ng/{url_item}'
   data = []
   pages = 50
   for x in range(1,pages + 1):
      url = f'{belt_list_url}/?page={x}#catalog-listing'

      response = requests.get(url)
      belt_items = BeautifulSoup(response.content,'html.parser')
   
      belt_product =  belt_items.find_all('article', class_='prd _fb col c-prd')

      for belt in belt_product:
        name = belt.find('h3', class_='name').text.strip()
        price = belt.find('div', class_='prc').text.strip()
        rating = belt.find('div', class_='rev')
        original_price = belt.find('div', class_='old')
          
        if url_item == "hair-accessories-8941":
           category = "women-hair-accessories"
        else:
           category = url_item
        if rating:
            rating = rating.text.strip()
        else:
            rating = 0
           
        if original_price:
            original_price = original_price.text.strip()
        else:
            original_price = 0
      
        data.append({
            'name': name,
            'price': price,
            'original_price':original_price,
            'rating': rating,
            'category': category      })
        

   df = pd.DataFrame(data)

def process_dataframe(df):
    # Define a regular expression pattern to extract rating and number of ratings
    pattern = r'([\d\.]+) out of 5\((\d+)\)'

    # Extract rating and number of ratings into new columns
    df[['Rating', 'Number of Ratings']] = df['rating'].str.extract(pattern)

    def get_min_price(text):
      if "-" in text:
        prices = text.split(" - ")
        # Take the average of the range
        price = sum(int(p.replace("₦", "").replace(",", "")) for p in prices) / len(prices)
      else:
        price = int(text.replace("₦", "").replace(",", ""))
      return int(price)
       
       
    df["price"] = df["price"].apply(get_min_price)

    df['original_price'] = df['original_price'].str.replace("₦", "").str.replace(",", "")
    df.drop(columns=['rating'], inplace=True)
    df.fillna(0, inplace=True)

    df['original_price'] = df['original_price'].astype(int)
    df['Rating'] = df['Rating'].astype(float)
    df['Number of Ratings'] = df['Number of Ratings'].astype(int)


    return df
process_file  = process_dataframe(df)
process_file.columns
print(process_file.columns) 